# Configuration
> Note: Input format is a bit cludgey, needs some pre-processing. This should be fixed in the next iteration.

In [56]:
input_file = 'hdfs:///files/salmonella/grouped_positions'
output_file = 'hdfs:///files/salmonella/assembly_reconstructed'

In [2]:
import findspark
findspark.init()
findspark.find()

'/usr/local/spark/python/pyspark'

In [3]:
from pyspark.sql import SparkSession

spark = (SparkSession
         .builder
         .master("yarn")
         .appName("python-testing")
         .config("spark.executor.instances", 16)
         .config("spark.executor.memory", "1536m")
         .getOrCreate())

sc = spark.sparkContext
sc

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
2022-04-24 10:38:12,635 WARN util.NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
2022-04-24 10:38:24,261 WARN yarn.Client: Neither spark.yarn.jars nor spark.yarn.archive is set, falling back to uploading libraries under SPARK_HOME.


<SparkContext master=yarn appName=python-testing>

In [57]:
raw = sc.textFile(input_file)
raw.take(1)

['"100000"\t["A", "A", "A", "A", "A", "A", "A", "A", "A", "A", "A"]']

In [58]:
kv = raw.map(lambda x: tuple(x.split('\t')))
kv.take(1)

[('"100000"', '["A", "A", "A", "A", "A", "A", "A", "A", "A", "A", "A"]')]

In [59]:
import json
pkv = kv.map(lambda x: (int(x[0][1:-1]), json.loads(x[1])))
pkv.take(1)

[(100000, ['A', 'A', 'A', 'A', 'A', 'A', 'A', 'A', 'A', 'A', 'A'])]

In [60]:
import collections
t = ["A", "A", "A", "B", "B", "B"]
collections.Counter(t).most_common(1)

[('A', 3)]

In [61]:
def mapFrequency(values):
    import collections
    
    counter = collections.Counter(values)
    prevalent_base_tuple = counter.most_common(1)[0]
    prevalent_base = prevalent_base_tuple[0]
    prevalent_base_count = prevalent_base_tuple[1]
    prevalent_base_confidence = prevalent_base_count / len(values)
    
    return (prevalent_base, prevalent_base_confidence, prevalent_base_count, counter.most_common())

assembled = pkv.mapValues(mapFrequency)
assembled.take(1)

[(100000, ('A', 1.0, 11, [('A', 11)]))]

In [64]:
assembled.saveAsPickleFile(output_file)